In [2]:
!pip install tensorflow
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.2 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.6 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 5.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [96]:
import os
import numpy as np
import random

import gym
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete

from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [97]:
ENV_NAME = 'TRIA-RL-MODEL3D'

In [104]:
TINI= 70.0; HINI= 40.0; AINI= 10.0 

TMIN =-40.0; TMAX=110; HMIN=0.0; HMAX=100.0; AMIN=0.0; AMAX=5000.0

EQUI_LEN= 60

R1= -0.25; R2= -0.5; R3= 2; NR3= -2

CONST_WAIGHT_VEC = [1,1,1,1]

D = {
     0 : [65.0, 80.0, 50.0, 85.0, 40.0, 90.0], 
     1 : [30.0, 50.0, 20.0, 60.0, 10.0, 70.0], 
     2 : [0.0, 19.0, 200.0, 599.0, 600.0, 2000.0]
    }

D1 = { 0: [65.0, 80.0], 1: [30.0, 50.0], 2: [0.0, 20.0]}

In [105]:
class TriaEnv(Env):
    
    def __init__(self):
        self.action_space = MultiDiscrete(np.array([2,2,2,2,2]))
        
        self.observation_space = Box(low=np.array([TMIN, HMIN, AMIN]), high=np.array([TMAX, HMAX, AMAX]))
        
        self.state = [TINI + random.uniform(-1.0,1.0), HINI + random.uniform(-1.0,1.0), AINI + random.uniform(-1.0,1.0)]
        
        #print('^^^', self.state, self.action_space)
        
        self.equilibrium_length = EQUI_LEN
        
    def step(self, action):
        
        ap = [1 if e == 1 else -1 for e in action]
        
        actionPrime = [a*b for a,b in zip(ap, CONST_WAIGHT_VEC)]
        
        actionAlgo = [actionPrime[a] - actionPrime[len(actionPrime) -a -1] for a in range(len(actionPrime) // 2)]
        
        actionAlgo.append(actionPrime[len(actionPrime) // 2])                                                              
        
        #print('***',actionAlgo, self.state)
        
        self.state = [a + b for a, b in zip(actionAlgo, self.state)]
        
        #print('&&&', actionAlgo, self.state)
        
        #reduce tria simulation length by 1 second
        self.equilibrium_length -= 1
        
        reward = [R3 if e >= D[i][0] and e<= D[i][1] else R2 if e >= D[i][2] and e<= D[i][3] else R1 if e >= D[i][4] and e <= D[i][5] else NR3 for i,e in enumerate(self.state)]
        #reward = [R3 if e >= D1[i][0] and e <= D1[i][1] else NR3  for i, e in enumerate(self.state)]

        reward = sum(reward)
        #print('$$$', reward)
            
        if self.equilibrium_length <= 0:
            terminated = True
        else:
            terminated = False
            
        info = {}
        #print('reward:{} state:{}'.format(reward, self.state))
        return self.state, reward, terminated, False, {}
    
    def render(self):
        pass
    
    def reset(self):
        
        self.state =[TINI + random.uniform(-1.0,1.0),HINI + random.uniform(-1.0,1.0),AINI + random.uniform(-1.0,1.0)]
        #print('@@@', self.state)
        self.equilibrium_length = EQUI_LEN
        
        return self.state, {}
        

In [106]:
env= TriaEnv()
print("1. Sample observation space: {}".format(env.observation_space.sample()))
print("2. Sample action space     : {}".format(env.action_space.sample()))
print("3. Sample state            : {}".format(env.state))

1. Sample observation space: [  95.159706   76.26532  2073.0872  ]
2. Sample action space     : [0 0 1 0 1]
3. Sample state            : [70.43901698547731, 39.449724215629956, 9.769497770457106]


In [107]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    #print(state)
    terminated = False
    score = 0 #[0,0,0] 
    
    while not terminated:
        #env.render()
        action = env.action_space.sample()
        #print(action, terminated , reward)
        #print(env.step(action))
        next_state, reward, terminated, truncated, info = env.step(action) 
        score += reward #[a + b for a, b in zip(reward, score)]
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:270.0
Episode:2 Score:325.0
Episode:3 Score:282.75
Episode:4 Score:215.0
Episode:5 Score:360


In [108]:
log_path = os.path.join('training','logs')
log_path

'training/logs'

In [109]:
#from stable_baselines3 import A2C
#from stable_baselines3.common.vec_env import SubprocVecEnv
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)
#model = A2C("MlpPolicy", env, verbose=1)

Using cpu device


In [110]:
model.learn(total_timesteps=1000)

2023-03-07 04:15:49.687392: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 04:15:56.716385: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-07 04:15:56.717291: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-07 04:15:56.717371: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Logging to training/logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 67   |
|    iterations      | 1    |
|    time_elapsed    | 30   |
|    total_timesteps | 2048 |
-----------------------------


In [111]:
Tria_Model_Path = os.path.join('training','save','Tria3DModel_PPO')

In [112]:
model.save(Tria_Model_Path)

In [113]:
del model

In [114]:
model = PPO.load(Tria_Model_Path, env=env)

In [115]:
evaluate_policy(model, env, n_eval_episodes=20, render=True)

/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:208: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(356.75, 14.16642156650719)

In [116]:
env.close()

In [117]:
episodes=10
for episode in range(1, episodes+1):
    observation = env.reset()
    terminated = False
    score = 0
    while not terminated:
        env.render()
        action, _ = model.predict(observation)
        observation, reward, terminated , info = env.step(action)
        score += reward
    print('Episone:{} Score:{}'.format( episode, score))
env.close()  

Episone:1 Score:[322.5]
Episone:2 Score:[348.]
Episone:3 Score:[277.25]
Episone:4 Score:[151.75]
Episone:5 Score:[287.5]
Episone:6 Score:[345.]
Episone:7 Score:[347.5]
Episone:8 Score:[340.]
Episone:9 Score:[313.25]
Episone:10 Score:[199.]


In [ ]:
!tensorboard --logdir './training/logs/' --bind_all  # training_log_path

2023-03-07 04:17:10.317233: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 04:17:11.777649: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 04:17:11.778269: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 04:17:17.878205: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-